## Pyspark Giriş

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
!pip install findspark

In [ ]:
import findspark

In [ ]:
findspark.init("/Users/krtdnc/spark/spark-3.5.0-bin-hadoop3")

In [ ]:
from pyspark import SparkContext

## Spark Uygulamasının Başlatılması

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [ ]:
spark = SparkSession.builder \
    .master("local") \
    .appName("pyspark_giris") \
    .getOrCreate()

sc = spark.sparkContext

In [ ]:
sc

In [ ]:
sc.version

In [ ]:
sc.appName

In [ ]:
sc.stop()

## Temel DataFrame İşlemleri

In [ ]:
sc

In [ ]:
spark_df = spark.read.csv("./churn.csv", header = True, inferSchema = True)

In [ ]:
spark_df.printSchema()

In [ ]:
type(spark_df)

In [ ]:
spark_df.cache()

In [ ]:
!pip install seaborn

In [ ]:
import seaborn as sns
df = sns.load_dataset("diamonds")

In [ ]:
type(df)

In [ ]:
df.head()

In [ ]:
spark_df.head() #Her pandas fonskiyonu spark_df lerde aynı çalışmaz ve farklı sonuçlar verebilir sparkın kendi fonksiyonlarını öğrenmek zorundayız.

In [ ]:
spark_df.show(5, truncate = True)

In [ ]:
spark_df.count()

In [ ]:
spark_df.columns

In [ ]:
spark_df.describe().show()

In [ ]:
spark_df.describe("Age").show()

In [ ]:
spark_df.select("Age","Names").show()

In [ ]:
spark_df.filter(spark_df.Age>40).count()

In [ ]:
spark_df.groupby("Churn").count().show()

In [ ]:
spark_df.groupby("Churn").agg({"Age":"mean"}).show()

In [ ]:
sc.stop()

## SQL İşlemleri

In [ ]:
sc

In [ ]:
spark_df.registerTempTable("tbl_df")

In [ ]:
spark.sql("show databases").show()

In [ ]:
spark.sql("show tables").show()

In [ ]:
spark.sql("select Age from tbl_df").show(5)

In [ ]:
spark.sql("select Churn, mean(Age) from tbl_df group by Churn").show()

In [ ]:
sc.stop()

## Büyük Veri Görselleştirme

In [ ]:
sc

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sdf = spark_df.toPandas() 

# Matplotlib pandas dataframe üzerinden çalışır bu yüzden önce spark ortamındaki dataframei pandas dataframeine çevirmemiz gerekiyor.
# Ancak çok büyük verilerde bunu yapamayacağımızdan dolayı sadece ihtiyacımız olan kolonları çekmemiz gerekir. Aşağıda örnek veriyoruz.
# Local makinamızda büyük veriyi görselleştiremeyeceğimiz için indirgeme işleminden sonra görselleştirmemiz gerekir. Örn: [46]

In [ ]:
sdf.head()

In [ ]:
sns.barplot(x = "Churn", y = sdf.Churn.index, data = sdf)

In [ ]:
a = spark_df.groupby("Churn").count().toPandas()

# Büyük veri indirgeme işlemi

In [ ]:
a

In [ ]:
sc.stop()

## Makine Öğrenmesi

In [ ]:
sc

In [ ]:
spark_df = spark.read.csv("./churn.csv", header = True, inferSchema = True)

In [ ]:
spark_df.show(5)

In [ ]:
spark_df = spark_df.toDF(*[c.lower() for c in spark_df.columns])

In [ ]:
spark_df.show(5)

In [ ]:
spark_df = spark_df.withColumnRenamed("_c0", "index")

In [ ]:
spark_df.show(5)

In [ ]:
spark_df.count()

In [ ]:
len(spark_df.columns)

In [ ]:
spark_df.columns

In [ ]:
spark_df.describe().show()

In [ ]:
spark_df.select("age","total_purchase","account_manager","years","num_sites","churn").describe().toPandas().transpose()

In [ ]:
spark_df = spark_df.dropna()

In [ ]:
spark_df = spark_df.withColumn("age_kare", spark_df.age**2)

In [ ]:
spark_df.show(5)

In [ ]:
# Bağımli değişkeni belirtme

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
stringIndexer = StringIndexer(inputCol = "churn", outputCol = "label")
mod = stringIndexer.fit(spark_df)
indexed = mod.transform(spark_df)
spark_df = indexed.withColumn("label", indexed["label"].cast("integer"))

In [ ]:
spark_df.show(5)

In [ ]:
# Bağımsız değişkenleri ayarlama

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
spark_df.columns

In [ ]:
bagimsiz_degiskenler = ["age", "total_purchase", "account_manager", "years", "num_sites"]

In [ ]:
vectorAssembler = VectorAssembler(inputCols = bagimsiz_degiskenler, outputCol = "features")
va_df = vectorAssembler.transform(spark_df)

In [ ]:
va_df.show(5)

In [ ]:
final_df = va_df.select(["features", "label"])

In [ ]:
final_df.show(5)

In [ ]:
# Test _ Train

In [ ]:
splits = final_df.randomSplit([0.70,0.30])
train_df = splits[0]
test_df = splits[1]

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
sc.stop()

## GBM ile Müşteri Terk Modellemesi

In [ ]:
sc

In [ ]:
from pyspark.ml.classification import GBTClassifier

In [ ]:
gbm = GBTClassifier(maxIter = 10, featuresCol = "features", labelCol = "label")

In [ ]:
gbm_model = gbm.fit(train_df)

In [ ]:
y_pred = gbm_model.transform(test_df)

In [ ]:
ac = y_pred.select("label", "prediction")

In [ ]:
ac.filter(ac.label == ac.prediction).count()/ac.count()

In [ ]:
# Model Tuning

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

evaluator = BinaryClassificationEvaluator()

paramGrid = (ParamGridBuilder()
             .addGrid(gbm.maxDepth, [2, 4, 6])
             .addGrid(gbm.maxBins, [20, 30])
             .addGrid(gbm.maxIter, [10, 20])
             .build())

cv = CrossValidator(estimator=gbm, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=10)

In [ ]:
cvModel = cv.fit(train_df)

In [ ]:
y_pred = cvModel.transform(test_df)

In [ ]:
ac = y_pred.select("label","prediction")

In [ ]:
ac.filter(ac.label == ac.prediction).count() / ac.count()

In [ ]:
# Yeni Müşteri Terk Eder mi ? Etmez mi ?

In [ ]:
import pandas as pd

names = pd.Series(["Ali Ahmetoğlu", "Taner Gün", "Berkay", "Polat Konak", "Kamil Atasoy"])
age = pd.Series([38, 43, 34, 50, 40])
total_purchase = pd.Series([30000, 10000, 6000, 30000, 100000])
account_manager = pd.Series([1, 0, 0, 1, 1])
years = pd.Series([20, 10, 3, 8, 30])
num_sites = pd.Series([30, 8, 8, 6, 50])

yeni_musteriler = pd.DataFrame({
    "names":names,
    "age":age,
    "total_purchase":total_purchase,
    "account_manager":account_manager,
    "years":years,
    "num_sites":num_sites})

yeni_musteriler.columns

In [ ]:
yeni_sdf = spark.createDataFrame(yeni_musteriler)

In [ ]:
yeni_sdf.show(5)

In [ ]:
yeni_musteriler = vectorAssembler.transform(yeni_sdf)

In [ ]:
results = cvModel.transform(yeni_musteriler)

In [ ]:
results.select("names","prediction").show()

In [ ]:
sc.stop()